In [1]:
import os
import gzip
import pandas as pd
import glob


pd.set_option('display.max_colwidth', None)

### 0. Decompress and save coverage

```python
import os
import gzip
import pandas as pd
import glob

input_dir = '/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Downloads/'
output_dir = '/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Decompress/'

# Get all file paths that end with .coverage.gz
file_paths = glob.glob(os.path.join(input_dir, '*.coverage.gz'))

# Traverse each file and store the results in a list
for i, file_path in enumerate(file_paths):
    # Construct the output file name
    output_file = os.path.join(output_dir, os.path.basename(file_path).replace('.coverage.gz', '_coverage.txt'))
    try:
        with gzip.open(file_path, 'rt') as f:
            lines = [line for line in f if not line.startswith('##')]
            data = [line.strip().split('\t') for line in lines[0:]]
            df = pd.DataFrame(data)
            df[['Position', 'N', 'Coverage']] = df.iloc[:, 0].str.split(',', expand=True)
            df = df.drop(df.columns[[0, 2]], axis=1)
            df['SUM'] = df['Coverage'].astype(int).sum()
            df['MEAN'] = df['SUM']/len(df)
            df['RATIO'] = df['Coverage'].astype(int)/df['MEAN'].astype(int)
            df = df.drop(['SUM','MEAN'], axis=1)
            # Save the results to a text file
            df.to_csv(output_file, sep='\t', header=True, index=False)
            # Delete the DataFrame to free up memory
            del df
    except gzip.BadGzipFile:
        print(f"Skipping {file_path}: gzip decompression failed.")
        continue
    except IndexError as e:
        file_id = os.path.basename(file_path)[:10]
        print(f"Error processing {file_id}: {e}")
        continue
    except EOFError as e:
        file_id = os.path.basename(file_path)[:10]
        print(f"Error processing {file_id}: {e}")
        continue
```